**Test for reading a single nanoDQMIO file directly from DAS and extracting histograms in a meaningful way**  
Note: with inspiration from here: https://github.com/cms-DQM/ML4DQM-DC_SharedTools/tree/master/dqmio

In [ ]:
### imports

import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import DQMIOReader
import importlib
importlib.reload(DQMIOReader)

In [ ]:
### get a file name from DAS using the client
# (not yet working, probably das client not installed)

#datasetname = '/MinimumBias/Commissioning2021-900GeVmkFit-v2/DQMIO'
#dascmd = "dasgoclient -query 'file dataset={}' --limit 0".format(datasetname)
#os.system(dascmd)

In [ ]:
### alternatively: define a file name from DAS manually
# (not yet working, probably need credentials and proxy)

#filename = '/store/data/Commissioning2021/MinimumBias/DQMIO/900GeVmkFit-v2/2810000/7C3F0F68-7951-11EC-A715-77D09A83BEEF.root'
#prefix = 'root://cms-xrd-global.cern.ch/'
#filename = prefix+filename

In [ ]:
### alternatively: define a local file
# (e.g. manual download from DAS on a system with credentials and proxy, then upload here)

filename = 'examplefile_nanoDQMIO_20220203.root'
#filename = '/eos/home-l/llambrec/Documents/ML4DQM-DC/temp_input_files_nanodqmio/examplefile_nanoDQMIO_store_data_Commissioning2021_MinimumBias_DQMIO_900GeVmkFit-v2_2810000_7C3F0F68-7951-11EC-A715-77D09A83BEEF.root'
#filename = '/eos/home-l/llambrec/Documents/ML4DQM-DC/temp_input_files_nanodqmio/examplefile_nanoDQMIO_store_data_Commissioning2021_MinimumBias_DQMIO_900GeVmkFit-v2_80000_03C9A2CC-607F-11EC-BBA5-0421A8C0BEEF.root'

In [ ]:
### read the file

reader = DQMIOReader.DQMIOReader(filename)

In [ ]:
### print the index (for internal testing, normally not needed)

rindex = reader.index
for key, val in rindex.items():
    # key is a tuple of (run number, lumisection number)
    # val is a list of IndexEntries (see DQMIOReader for details)
    print(key)
    for el in val: print('  '+str(el))

In [ ]:
### print all available monitoring elements

melist = reader.melist
print('Available histograms ({}):'.format(len(melist.keys())))
for me,tp in melist.items(): print('- {} (type {})'.format(me,DQMIOReader.DQMIOReader.getMEType(tp)))

In [ ]:
### print all available lumisections

runsls = reader.listLumis()
print('Available lumisections: ({})'.format(len(runsls)))
for runls in runsls: print('- Run {}, LS {}'.format(runls[0],runls[1]))

In [ ]:
### retrieve all instances of a given histogram type

histname = 'PixelPhase1/Tracks/PXBarrel/chargeInner_PXLayer_1'
mes = reader.getSingleMEs(histname)
for me in mes: print(me)

In [ ]:
### retrieve all instances of a collection of histogram types

histname = 'PixelPhase1/Tracks/PXBarrel/chargeInner_PXLayer_?'
mes = reader.getMEs(histname)
for me in mes: print(me)

In [ ]:
### plot a histogram

histname = 'PixelPhase1/Tracks/PXBarrel/chargeInner_PXLayer_1'
lumi = (346448,1)
me = reader.getSingleMEForLumi(lumi,histname)
hist = me.data
histarray = np.zeros(hist.GetNbinsX()+2)
for i in range(0,hist.GetNbinsX()+2):
    histarray[i] = hist.GetBinContent(i)
fig,ax = plt.subplots()
ax.plot(histarray)

In [ ]:
### plot multiple histograms

histname = 'PixelPhase1/Tracks/PXBarrel/chargeInner_PXLayer_1'
mes = reader.getSingleMEs(histname)
histarray = np.zeros((len(mes),mes[0].data.GetNbinsX()+2))
for i,me in enumerate(mes):
    for j in range(0,me.data.GetNbinsX()+2):
        histarray[i,j] = me.data.GetBinContent(j)
fig,ax = plt.subplots()
for hist in histarray: ax.plot(hist)